<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2
import os
from dotenv import load_dotenv
from bs4 import BeautifulSoup
import requests
import warnings

# Убрать предупреждения по (не)использованию SQLAlchemy
warnings.filterwarnings('ignore', category=UserWarning)
# загрузка переменных для доступа к БД
load_dotenv()

True

In [2]:
connection = psycopg2.connect(
    dbname=os.getenv("DBNAME"),
    user=os.getenv("DBUSER"),
    host=os.getenv("HOST"),
    password=os.getenv("PASSWORD"),
    port=os.getenv("PORT")
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [3]:
query = f'''
SELECT COUNT(id) count
FROM vacancies
'''

In [4]:
result = pd.read_sql_query(query, connection)
print(f"Количество вакансий в базе: {result["count"].iloc[0]}")

Количество вакансий в базе: 49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [5]:
query = f'''
SELECT COUNT(id) count
FROM employers
'''

In [6]:
result = pd.read_sql_query(query, connection)
print(f"Количество работодателей: {result["count"].iloc[0]}")

Количество работодателей: 23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [7]:
query = f'''
SELECT COUNT(id) count
FROM areas
'''

In [8]:
result = pd.read_sql_query(query, connection)
print(f"Количество регионов: {result["count"].iloc[0]}")

Количество регионов: 1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [9]:
query = f'''
SELECT COUNT(id) count
FROM industries
'''

In [10]:
result = pd.read_sql_query(query, connection)
print(f"Количество сфер деятельности: {result["count"].iloc[0]}")

Количество сфер деятельности: 294


***

#### Предварительная информация о данных:
- У нас есть около 50к вакансий, которые предлагают 23к работодателей. В среднем имеется 2 вакансии на работодателя
- Данные по вакансиям нам предложены из 1362 регионов и 294 сфер деятельности.

_Судя по всему, данные категоризированы, требуется дальнейшее изучение._


# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [11]:
query = f'''
SELECT
	a.name "area",
	COUNT(*) cnt
FROM
	areas a
LEFT JOIN vacancies v ON
	v.area_id = a.id
GROUP BY
	a.name
ORDER BY
	cnt DESC
LIMIT 5
'''

In [12]:
result = pd.read_sql_query(query, connection)
print(f"Пятёрка лидеров по количеству вакансий: "
      f"{', '.join(result["area"].tolist())}")

Пятёрка лидеров по количеству вакансий: Москва, Санкт-Петербург, Минск, Новосибирск, Алматы


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [13]:
query = f'''
SELECT
	count(*)
FROM
	vacancies
WHERE
	salary_from IS NOT NULL
	OR salary_to IS NOT NULL
'''

In [14]:
result = pd.read_sql_query(query, connection)
print(f"Количество вакансий, где заполнено хотя бы одно из двух полей с зарплатой: "
      f"{result["count"].iloc[0]}")

Количество вакансий, где заполнено хотя бы одно из двух полей с зарплатой: 24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [15]:
query = f'''
SELECT
	round(avg(salary_from)) avg_salary_from,
	round(avg(salary_to)) avg_salary_to
FROM
	vacancies
'''

In [16]:
result = pd.read_sql_query(query, connection)
print(f"Среднее значения для нижней границы зарплатной вилки: {result["avg_salary_from"].iloc[0]}")
print(f"Среднее значения для верхней границы зарплатной вилки: {result["avg_salary_to"].iloc[0]}")

Среднее значения для нижней границы зарплатной вилки: 71065.0
Среднее значения для верхней границы зарплатной вилки: 110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [17]:
query = f'''
SELECT
	schedule,
	employment,
	count(*) "cnt"
FROM
	vacancies
GROUP BY
	schedule ,
	employment
ORDER BY
	cnt DESC
'''

In [18]:
result = pd.read_sql_query(query, connection)
print(f"Самое популярное сочетание раб. графика и типа трудоустройства: "
      f"{result['schedule'].iloc[1], result['employment'].iloc[1]}")

Самое популярное сочетание раб. графика и типа трудоустройства: ('Удаленная работа', 'Полная занятость')


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [19]:
query = f'''
SELECT
	experience,
	count(*) cnt
FROM
	vacancies
GROUP BY
	experience
ORDER BY
	cnt
'''

In [20]:
result = pd.read_sql_query(query, connection)
print(result)

           experience    cnt
0         Более 6 лет   1337
1           Нет опыта   7197
2       От 3 до 6 лет  14511
3  От 1 года до 3 лет  26152


***

#### Выводы по детальному анализу вакансий
- Основные заказчики расположены в Москве и Питере. А также у нас много вакансий в городе Минск
- Большая часть соискателей ищет удаленный формат работы на полный день
- Почти половина вакансий не имеет данных по желаемой заработной плате
- В базе много вакансий для начинающих специалистов

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [21]:
query = f'''
(SELECT
	e.name employer,
	count(*) cnt
FROM
	vacancies v
LEFT JOIN employers e ON
	e.id = v.employer_id
GROUP BY
	e.name
ORDER BY
	cnt DESC
LIMIT 1)

UNION

(SELECT
	e.name employer,
	count(*) cnt
FROM
	vacancies v
LEFT JOIN employers e ON
	e.id = v.employer_id
GROUP BY
	e.name
ORDER BY
	cnt DESC
LIMIT 1 OFFSET 4)

ORDER BY
	cnt DESC
'''

In [22]:
result = pd.read_sql_query(query, connection)
print(f"первое место по вакансиям: "
      f"{result['employer'].iloc[0]}")
print(f"пятое место по вакансиям: "
      f"{result['employer'].iloc[1]}")

первое место по вакансиям: Яндекс
пятое место по вакансиям: Газпром нефть


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [23]:
query = f'''
SELECT
	a.name,
	COUNT(e.id) cnt
FROM
	employers e
LEFT JOIN
    areas a ON
	a.id = e.area
LEFT JOIN
    vacancies v ON
	v.area_id = a.id
WHERE
	v.id IS NULL
GROUP BY
	a.name
ORDER BY
	cnt DESC

'''

In [24]:
result = pd.read_sql_query(query, connection)
print(f"Регион, где нет вакансий, но большее количеством работодателей: "
      f"{result['name'].iloc[0]}")

Регион, где нет вакансий, но большее количеством работодателей: Россия


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества. Выберите максимальное значение из получившегося списка.


In [25]:
query = f'''
SELECT
	employer_id emp,
	count(DISTINCT area_id) cnt
FROM
	vacancies
GROUP BY
	employer_id
ORDER BY
	cnt DESC
'''

In [26]:
result = pd.read_sql_query(query, connection)
print(f"Количество регионов, в которых публикуется самый активный работодатель: "
      f"{result['cnt'].iloc[0]}")

Количество регионов, в которых публикуется самый активный работодатель: 181


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности.

In [27]:
query = f'''
WITH no_info_about_industries AS (
SELECT
	e.id emp_id,
	count(ei.industry_id) cnt
FROM
	employers e
LEFT JOIN employers_industries ei ON
	e.id = ei.employer_id
GROUP BY
	emp_id
HAVING
	count(ei.industry_id) = 0
ORDER BY
	cnt)

SELECT
	count(*) cnt
FROM
	no_info_about_industries
'''

In [28]:
result = pd.read_sql_query(query, connection)
print(f"Количество работодателей, у которых не указана сфера деятельности: "
      f"{result['cnt'].iloc[0]}")

Количество работодателей, у которых не указана сфера деятельности: 8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [29]:
query = f'''
SELECT
	e.name emp_name,
	count(ei.industry_id) cnt
FROM
	employers e
LEFT JOIN employers_industries ei ON
	e.id = ei.employer_id
GROUP BY
	e.id
HAVING
	count(ei.industry_id) = 4
ORDER BY
	e.name
LIMIT 1 OFFSET 2
'''

In [30]:
result = pd.read_sql_query(query, connection)
print(f"Компания, которая находится на третьем месте, среди компаний у которых указано 4 сферы деятельности: "
      f"{result['emp_name'].iloc[0]}")

Компания, которая находится на третьем месте, среди компаний у которых указано 4 сферы деятельности: 2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [31]:
query = f'''
SELECT
	count(*) cnt
FROM
	employers e
LEFT JOIN employers_industries ei ON
	e.id = ei.employer_id
WHERE
	ei.industry_id = (
	SELECT
		id
	FROM
		industries
	WHERE
		name = 'Разработка программного обеспечения')
'''

In [32]:
result = pd.read_sql_query(query, connection)
print(f"Количество компаний со сферой деятельности 'Разработка программного обеспечения': "
      f"{result['cnt'].iloc[0]}")

Количество компаний со сферой деятельности 'Разработка программного обеспечения': 3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [33]:
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
wiki_page = requests.get(url)
if wiki_page.status_code == 200:
    soup = BeautifulSoup(wiki_page.text, "html.parser") # получаем в нужном формате для разбора страницу
    # таблицу можно найти по классам, которые она использует
    table_city = soup.find("table",
                           class_=["standard",  "sortable",  "jquery-tablesorter"])
    cells = [] # в эту переменную сохраним значения ячеек из таблицы
    for row in table_city.find_all("tr")[1:]: # пропускам заголовок
        cells.append([td.text.strip() for td in row.find_all("td")])

    cities = tuple(each[1] for each in cells) # во втором столбце содержится название города-миллионника. Сохраним в виде кортежа

In [34]:
query = f'''
SELECT
	e.name emp,
	a.name city,
	count(*) cnt
FROM
	vacancies v
LEFT JOIN employers e ON
	v.employer_id = e.id
LEFT JOIN areas a ON
	a.id = v.area_id
WHERE
	e.name = 'Яндекс'
	AND a.name IN {cities}
GROUP BY
	e.name,
	a.name

UNION

SELECT
	'Яндекс',
	'Total',
	count(*) cnt
FROM
	vacancies v
LEFT JOIN employers e ON
	v.employer_id = e.id
LEFT JOIN areas a ON
	a.id = v.area_id
WHERE
	e.name = 'Яндекс'
	AND a.name IN {cities}

ORDER BY 3
'''

In [35]:
result = pd.read_sql_query(query, connection)
print(f"Количество строк в итоговой таблице: "
      f"{result.shape[0]}")
print(f"Результат в строке Total: "
      f"{result['cnt'].iloc[result.shape[0] - 1]}")

Количество строк в итоговой таблице: 17
Результат в строке Total: 485


***

#### Выводы по детальному анализу работодателей
- Огромное количество вакансий от компании Яндекс, которая ко количеству размещаемых данных значительно обходит другие компании
- Причем бОльшая часть их вакансий размещена не в городах-миллионниках
- При этом у нас в базе всего не так много компаний с указанной сферой деятельности "Разработка программного обеспечения"

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [36]:
query = f'''
SELECT
	count(*) cnt
FROM
	vacancies v
WHERE
	LOWER(v.name) LIKE '%data%'
	OR LOWER(v.name) LIKE '%данн%'
'''

In [37]:
result = pd.read_sql_query(query, connection)
print(f"Количество вакансий, которые имеют отношение к данным: {result["cnt"].iloc[0]}")

Количество вакансий, которые имеют отношение к данным: 1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [38]:
query = f'''
SELECT
	count(*) cnt
FROM
	vacancies
WHERE
	((LOWER(name) LIKE '%data scientist%'
		OR LOWER(name) LIKE '%data science%'
			OR LOWER(name) LIKE '%исследователь данных%'
				OR LOWER(name) LIKE '%machine learning%'
					OR LOWER(name) LIKE '%машинн%обучен%'
						OR name LIKE '%ML%')
	AND LOWER(name) NOT LIKE '%html%')
	AND(LOWER(name) LIKE '%junior%'
		OR experience = 'Нет опыта'
		OR schedule = 'Стажировка')
'''

In [39]:
result = pd.read_sql_query(query, connection)
print(f"Количество вакансий, которые подходят для начинающего дата-сайентиста: {result["cnt"].iloc[0]}")

Количество вакансий, которые подходят для начинающего дата-сайентиста: 51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [40]:
query = f'''
SELECT
	count(*) cnt
FROM
	vacancies
WHERE
	(name ILIKE '%data scientist%'
		OR name ILIKE '%data science%'
		OR name ILIKE '%исследователь данных%'
		OR name ILIKE '%ml%'
		AND name NOT LIKE '%HTML%'
		AND name NOT LIKE '%QML%'
		OR name ILIKE '%machine learning%'
		OR name ILIKE '%машинн%обучен%')
	AND (key_skills ILIKE '%sql%'
		OR key_skills ILIKE '%postgres%')
'''

In [41]:
result = pd.read_sql_query(query, connection)
print(f"Количество вакансий, с ключевым навыком SQL или Postgres: {result["cnt"].iloc[0]}")

Количество вакансий, с ключевым навыком SQL или Postgres: 229


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [42]:
query = f'''
SELECT
	count(*) cnt
FROM
	vacancies
WHERE
	(name ILIKE '%data scientist%'
		OR name ILIKE '%data science%'
		OR name ILIKE '%исследователь данных%'
		OR name ILIKE '%ml%'
		AND name NOT LIKE '%HTML%'
		AND name NOT LIKE '%QML%'
		OR name ILIKE '%machine learning%'
		OR name ILIKE '%машинн%обучен%')
	AND key_skills ILIKE '%python%'
'''

In [43]:
result = pd.read_sql_query(query, connection)
print(f"Количество вакансий, с ключевым навыком python: {result["cnt"].iloc[0]}")

Количество вакансий, с ключевым навыком python: 357


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [44]:
query = f'''
SELECT
	ROUND(AVG(LENGTH(key_skills) - LENGTH(REPLACE(key_skills, CHR(9), '')) + 1),
	2) avg
FROM
	vacancies
WHERE
	(name ILIKE '%data scientist%'
		OR name ILIKE '%data science%'
		OR name ILIKE '%исследователь данных%'
		OR name LIKE '%ML%'
		AND name NOT ILIKE '%html%'
		OR name ILIKE '%machine learning%'
		OR name ILIKE '%машинн%обучен%')
	AND key_skills IS NOT NULL
'''

In [45]:
result = pd.read_sql_query(query, connection)
print(f"Среднее количество ключевых навыков: {result["avg"].iloc[0]}")

Среднее количество ключевых навыков: 6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [46]:
query = f'''
SELECT
	experience exp,
	ROUND(AVG(COALESCE((salary_from + salary_to) / 2, salary_from, salary_to)),
	0) zp
FROM
	vacancies
WHERE
	(name ILIKE '%data scientist%'
		OR name ILIKE '%data science%'
		OR name ILIKE '%исследователь данных%'
		OR name LIKE '%ML%'
		AND name NOT ILIKE '%html%'
		OR name ILIKE '%machine learning%'
		OR name ILIKE '%машинн%обучен%')
GROUP BY experience
'''

In [47]:
result = pd.read_sql_query(query, connection)
print(f"Средняя зарплата для дата-саентистов с опытом от 3 до 6 лет: {result[result["exp"] == 'От 3 до 6 лет']['zp'].iloc[0]}")

Средняя зарплата для дата-саентистов с опытом от 3 до 6 лет: 243115.0


***

#### Выводы по детальному анализу вакансий
- Представлено очень малое количество вакансий для начинающих дата-сайентистов
- Среди 1771 вакансий связанных с данными только 229 требует знания SQL как ключевого навыка
- В целом, профессия Data Science подразумевает знания 6-7 ключевых навыков

# Общий вывод по проекту

- Представлено большое количество по профессии "Системный администратор" и по направлению 1С
- Традиционно, большее количество вакансий представлено в Москве и Санкт-Петербурге
- У нас есть работодатели с совпадающими именами. Самые популярные "Новые Технологии" и "Перспектива"

В дальнейшем можно исследовать количество ИТ-вакансий в компаниях, где профиль деятельности не ИТ

